# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

0. KarcıFANN FER2013 veriseti ile...  Ilk olarak dataseti indirelim


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

1) Imports + yardımcı fonksiyonlar

In [4]:
import os, cv2, numpy as np, mediapipe as mp
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

# -----------------------------
# Helpers
# -----------------------------
def softmax(z):
    z = z - np.max(z, axis=1, keepdims=True)
    e = np.exp(z)
    return e / np.sum(e, axis=1, keepdims=True)

def onehot(y_idx, num_classes):
    return np.eye(num_classes, dtype=np.float32)[y_idx]

def l2(x):
    return float(np.sqrt(np.sum(x * x)))

def majority_vote(items, k=15):
    if len(items) == 0:
        return None
    last = items[-k:]
    vals, cnts = np.unique(last, return_counts=True)
    return vals[np.argmax(cnts)]

def compute_class_weights(y, num_classes):
    # inverse frequency (normalize mean=1)
    counts = np.bincount(y, minlength=num_classes).astype(np.float32)
    w = 1.0 / np.maximum(counts, 1.0)
    w = w * (num_classes / np.sum(w))
    return w.astype(np.float32)

def weighted_cross_entropy(y_pred, y_true_oh, class_w=None, eps=1e-9):
    y_pred = np.clip(y_pred, eps, 1 - eps)
    logp = np.log(y_pred)
    per = -np.sum(y_true_oh * logp, axis=1)  # (batch,)
    if class_w is not None:
        sw = np.sum(y_true_oh * class_w.reshape(1, -1), axis=1)  # sample weight
        per = per * sw
    return float(np.mean(per))

mp_face_mesh = mp.solutions.face_mesh
mp_face_detection = mp.solutions.face_detection


In [30]:
def softmax(z):
    z = z - np.max(z, axis=1, keepdims=True)
    e = np.exp(z)
    return e / (np.sum(e, axis=1, keepdims=True) + 1e-12)

def onehot(y_idx, num_classes):
    return np.eye(num_classes, dtype=np.float32)[y_idx]

def l2(x):
    x = np.asarray(x, dtype=np.float32)
    return float(np.sqrt(np.sum(x * x) + 1e-12))

def majority_vote(items, k=15):
    if len(items) == 0:
        return None
    last = items[-k:]
    vals, cnts = np.unique(last, return_counts=True)
    return vals[np.argmax(cnts)]


2) MediaPipe FaceMesh (face_only) stabil pipeline

In [31]:
mp_face_mesh = mp.solutions.face_mesh
mp_face_detection = mp.solutions.face_detection
mp_holistic = mp.solutions.holistic

FACE_LM_TARGET = 468  # hedef 468 landmark (refine_landmarks=False)

# -----------------------------
# Preprocess (FER küçük/kontrast düşük)
# -----------------------------
def preprocess_for_facemesh(bgr, resize_to=384, use_clahe=True, use_sharpen=True):
    img = bgr
    if resize_to is not None:
        img = cv2.resize(img, (resize_to, resize_to), interpolation=cv2.INTER_CUBIC)

    if use_clahe:
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        l2 = clahe.apply(l)
        lab = cv2.merge([l2, a, b])
        img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    if use_sharpen:
        blur = cv2.GaussianBlur(img, (0, 0), 1.2)
        img = cv2.addWeighted(img, 1.5, blur, -0.5, 0)

    return img

def detect_face_bbox(face_det, rgb, w, h, margin=0.25):
    det = face_det.process(rgb)
    if not det.detections:
        return None, 0.0
    best = max(det.detections, key=lambda d: d.score[0] if d.score else 0.0)
    score = float(best.score[0]) if best.score else 0.0
    bbox = best.location_data.relative_bounding_box

    x1 = int(bbox.xmin * w); y1 = int(bbox.ymin * h)
    bw = int(bbox.width * w); bh = int(bbox.height * h)
    x2 = x1 + bw; y2 = y1 + bh

    mx = int(bw * margin); my = int(bh * margin)
    x1 = max(0, x1 - mx); y1 = max(0, y1 - my)
    x2 = min(w - 1, x2 + mx); y2 = min(h - 1, y2 + my)

    if x2 <= x1 or y2 <= y1:
        return None, 0.0
    return (x1, y1, x2, y2), score

def _unique_indices_from_connections(conns):
    s = set()
    for a, b in conns:
        s.add(a); s.add(b)
    return np.array(sorted(list(s)), dtype=np.int32)

LEFT_EYE_IDX  = _unique_indices_from_connections(mp_face_mesh.FACEMESH_LEFT_EYE)
RIGHT_EYE_IDX = _unique_indices_from_connections(mp_face_mesh.FACEMESH_RIGHT_EYE)

def align_landmarks_xyzn(pts_xyz, score, eps=1e-6):
    """
    pts_xyz: (468,3) in pixel-like coordinates (x,y) and z arbitrary scale
    returns: (468,4) -> aligned x,y,z plus v=score
    """
    L = pts_xyz[LEFT_EYE_IDX, :2].mean(axis=0)
    R = pts_xyz[RIGHT_EYE_IDX, :2].mean(axis=0)

    dx, dy = (R - L).astype(np.float32)
    dist = float(np.sqrt(dx*dx + dy*dy) + eps)
    ang = float(np.arctan2(dy, dx))

    c, s = np.cos(-ang), np.sin(-ang)
    rot = np.array([[c, -s],[s, c]], dtype=np.float32)

    mid = (L + R) / 2.0
    xy = pts_xyz[:, :2] - mid.reshape(1, 2)
    xy = (xy @ rot.T) / dist

    z = pts_xyz[:, 2] / dist  # scale z similarly
    v = np.full((pts_xyz.shape[0], 1), float(score), dtype=np.float32)

    out = np.concatenate([xy.astype(np.float32), z.reshape(-1,1).astype(np.float32), v], axis=1)
    return out

def facemesh_features_from_bgr(face_det, face_mesh, bgr, resize_to=384):
    """
    Output: 468*4 = 1872 feature (aligned + score)
    Tries multiple ways to reduce skip:
      1) preprocessed resize
      2) face crop with FaceDetection
      3) full frame fallback
      4) multiscale fallback
    """
    # multi-scale attempts (büyük -> küçük)
    for rt in [resize_to, 512, 384, 256]:
        if rt is None:
            continue

        img = preprocess_for_facemesh(bgr, resize_to=rt, use_clahe=True, use_sharpen=True)
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = rgb.shape[:2]

        bbox, score = detect_face_bbox(face_det, rgb, w, h, margin=0.30)

        # candidate 1: crop
        if bbox is not None:
            x1,y1,x2,y2 = bbox
            crop = rgb[y1:y2, x1:x2]
            if crop.size > 0:
                res = face_mesh.process(crop)
                if res.multi_face_landmarks:
                    lm = res.multi_face_landmarks[0].landmark
                    pts = np.zeros((468,3), dtype=np.float32)
                    # landmark x,y are normalized to crop
                    cw = max(1, crop.shape[1]); ch = max(1, crop.shape[0])
                    for i, p in enumerate(lm[:468]):  # safeguard
                        pts[i,0] = (p.x * cw) + x1
                        pts[i,1] = (p.y * ch) + y1
                        pts[i,2] = p.z  # z already relative
                    aligned = align_landmarks_xyzn(pts, score)
                    return aligned.reshape(-1)

        # candidate 2: full frame fallback
        res2 = face_mesh.process(rgb)
        if res2.multi_face_landmarks:
            lm = res2.multi_face_landmarks[0].landmark
            pts = np.zeros((468,3), dtype=np.float32)
            for i, p in enumerate(lm[:468]):
                pts[i,0] = p.x * w
                pts[i,1] = p.y * h
                pts[i,2] = p.z
            aligned = align_landmarks_xyzn(pts, score)
            return aligned.reshape(-1)

    return None

def expected_feature_dim(mode="face_only"):
    if mode != "face_only":
        raise ValueError("Bu pipeline sadece face_only için optimize edildi.")
    return 468 * 4

def extract_fer2013_folder(root, out="fer_mp_align.npz", resize_to=384, max_per_class=None):
    train_dir = os.path.join(root, "train")
    test_dir  = os.path.join(root, "test")
    if not os.path.isdir(train_dir) or not os.path.isdir(test_dir):
        raise RuntimeError(f"train/test klasörleri bulunamadı: {train_dir} , {test_dir}")

    classes = sorted([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))])
    if not classes:
        raise RuntimeError("train içinde class klasörleri bulunamadı.")

    dim = expected_feature_dim("face_only")
    print("Classes:", classes)

    def list_images(folder):
        imgs = [f for f in os.listdir(folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        imgs.sort()
        if max_per_class is not None:
            imgs = imgs[:max_per_class]
        return imgs

    Xtr, ytr, Xte, yte = [], [], [], []
    skipped = {"train": 0, "test": 0}
    used = {"train": 0, "test": 0}

    with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.15) as face_det, \
         mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1,
                               refine_landmarks=False,
                               min_detection_confidence=0.15,
                               min_tracking_confidence=0.15) as face_mesh:

        def process_split(split_name, split_path, X_list, y_list):
            for ci, cls in enumerate(classes):
                folder = os.path.join(split_path, cls)
                imgs = list_images(folder)
                print(f"[{split_name}] {cls}: {len(imgs)} görüntü")

                for fn in imgs:
                    p = os.path.join(folder, fn)
                    img = cv2.imread(p)
                    if img is None:
                        skipped[split_name] += 1
                        continue

                    feat = facemesh_features_from_bgr(face_det, face_mesh, img, resize_to=resize_to)
                    if feat is None or feat.shape[0] != dim:
                        skipped[split_name] += 1
                        continue

                    X_list.append(feat)
                    y_list.append(ci)
                    used[split_name] += 1

        process_split("train", train_dir, Xtr, ytr)
        process_split("test",  test_dir,  Xte, yte)

    if len(Xtr) == 0 or len(Xte) == 0:
        raise RuntimeError("Hiç veri çıkmadı. resize_to=512 dene veya dataset yolunu kontrol et.")

    X_train = np.stack(Xtr, axis=0).astype(np.float32)
    y_train = np.array(ytr, dtype=np.int64)
    X_test  = np.stack(Xte, axis=0).astype(np.float32)
    y_test  = np.array(yte, dtype=np.int64)

    np.savez_compressed(
        out,
        X_train=X_train, y_train=y_train,
        X_test=X_test,   y_test=y_test,
        classes=np.array(classes),
        mode="face_only"
    )

    print("\nSaved:", out)
    print("Train:", X_train.shape, "Test:", X_test.shape)
    print("Used:", used, "Skipped:", skipped)

def detect_face_bbox(face_det, rgb, w, h, margin=0.25):
    det = face_det.process(rgb)
    if not det.detections:
        return None, 0.0
    best = max(det.detections, key=lambda d: d.score[0] if d.score else 0.0)
    score = float(best.score[0]) if best.score else 0.0
    bbox = best.location_data.relative_bounding_box

    x1 = int(bbox.xmin * w)
    y1 = int(bbox.ymin * h)
    bw = int(bbox.width * w)
    bh = int(bbox.height * h)
    x2 = x1 + bw
    y2 = y1 + bh

    mx = int(bw * margin)
    my = int(bh * margin)
    x1 = max(0, x1 - mx)
    y1 = max(0, y1 - my)
    x2 = min(w - 1, x2 + mx)
    y2 = min(h - 1, y2 + my)

    if x2 <= x1 or y2 <= y1:
        return None, 0.0
    return (x1, y1, x2, y2), score

def facemesh_features_from_bgr(face_det, face_mesh, bgr, resize_to=384):
    """
    468*(x,y,z,score)=1872 boyut.
    - bbox crop + full frame fallback
    - 478 gelirse otomatik 468'e kırpma/padding (IndexError biter)
    """
    img = preprocess_for_facemesh(bgr, resize_to=resize_to, use_clahe=True, use_sharpen=True)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = rgb.shape[:2]

    bbox, score = detect_face_bbox(face_det, rgb, w, h, margin=0.25)

    candidates = []
    if bbox is not None:
        x1, y1, x2, y2 = bbox
        crop = rgb[y1:y2, x1:x2]
        if crop.size > 0:
            candidates.append((face_mesh.process(crop), score))

    candidates.append((face_mesh.process(rgb), score))

    for res, sc in candidates:
        if res.multi_face_landmarks:
            lm = res.multi_face_landmarks[0].landmark
            n = len(lm)

            feat = np.zeros((FACE_LM_TARGET, 4), dtype=np.float32)
            use_n = min(FACE_LM_TARGET, n)
            for i in range(use_n):
                p = lm[i]
                feat[i, 0] = float(p.x)
                feat[i, 1] = float(p.y)
                feat[i, 2] = float(p.z)
                feat[i, 3] = float(sc)

            return feat.reshape(-1)

    return None

def expected_feature_dim(mode):
    dim = 468 * 4
    if mode in ("face_pose", "face_pose_hands"):
        dim += 33 * 4
    if mode == "face_pose_hands":
        dim += 21 * 4 + 21 * 4
    return dim


3) Holistic (opsiyonel modlar için)

In [32]:
def _lm4(lm):
    v = getattr(lm, "visibility", None)
    if v is None:
        v = getattr(lm, "presence", 0.0)
    return [float(lm.x), float(lm.y), float(lm.z), float(v)]

def extract_features_from_results(results, mode="face_only"):
    parts = []
    want_pose = mode in ("face_pose", "face_pose_hands")
    want_hands = mode == "face_pose_hands"

    if results.face_landmarks is not None:
        face = results.face_landmarks.landmark
        parts.append(np.array([_lm4(lm) for lm in face], dtype=np.float32).flatten())
    else:
        parts.append(np.zeros((468 * 4,), dtype=np.float32))

    if want_pose:
        if results.pose_landmarks is not None:
            pose = results.pose_landmarks.landmark
            parts.append(np.array([_lm4(lm) for lm in pose], dtype=np.float32).flatten())
        else:
            parts.append(np.zeros((33 * 4,), dtype=np.float32))

    if want_hands:
        if results.left_hand_landmarks is not None:
            lh = results.left_hand_landmarks.landmark
            parts.append(np.array([_lm4(lm) for lm in lh], dtype=np.float32).flatten())
        else:
            parts.append(np.zeros((21 * 4,), dtype=np.float32))

        if results.right_hand_landmarks is not None:
            rh = results.right_hand_landmarks.landmark
            parts.append(np.array([_lm4(lm) for lm in rh], dtype=np.float32).flatten())
        else:
            parts.append(np.zeros((21 * 4,), dtype=np.float32))

    return np.concatenate(parts, axis=0).astype(np.float32)


4) Dataset extract (fer2013 klasöründen npz)

In [33]:
def extract_fer2013_folder(root, mode="face_only", out="fer_mp.npz", resize_to=384, max_per_class=None):
    train_dir = os.path.join(root, "train")
    test_dir  = os.path.join(root, "test")

    if not os.path.isdir(train_dir) or not os.path.isdir(test_dir):
        raise RuntimeError(f"train/test klasörleri bulunamadı: {train_dir} , {test_dir}")

    classes = sorted([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))])
    if not classes:
        raise RuntimeError("train içinde class klasörleri bulunamadı.")

    print("Classes:", classes)
    dim = expected_feature_dim(mode)

    Xtr, ytr, Xte, yte = [], [], [], []
    skipped = {"train": 0, "test": 0}
    used = {"train": 0, "test": 0}

    def list_images(folder):
        imgs = [f for f in os.listdir(folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        imgs.sort()
        if max_per_class is not None:
            imgs = imgs[:max_per_class]
        return imgs

    if mode == "face_only":
        with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.2) as face_det, \
             mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1,
                                   refine_landmarks=False,
                                   min_detection_confidence=0.2,
                                   min_tracking_confidence=0.2) as face_mesh:

            def process_split(split_name, split_path, X_list, y_list):
                for ci, cls in enumerate(classes):
                    folder = os.path.join(split_path, cls)
                    imgs = list_images(folder)
                    print(f"[{split_name}] {cls}: {len(imgs)} görüntü")

                    for fn in imgs:
                        p = os.path.join(folder, fn)
                        img = cv2.imread(p)
                        if img is None:
                            skipped[split_name] += 1
                            continue

                        feat = facemesh_features_from_bgr(face_det, face_mesh, img, resize_to=resize_to)
                        if feat is None or feat.shape[0] != dim:
                            skipped[split_name] += 1
                            continue

                        X_list.append(feat)
                        y_list.append(ci)
                        used[split_name] += 1

            process_split("train", train_dir, Xtr, ytr)
            process_split("test",  test_dir,  Xte, yte)

    else:
        with mp_holistic.Holistic(
            static_image_mode=True,
            model_complexity=1,
            refine_face_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        ) as holistic:

            def process_split(split_name, split_path, X_list, y_list):
                for ci, cls in enumerate(classes):
                    folder = os.path.join(split_path, cls)
                    imgs = list_images(folder)
                    print(f"[{split_name}] {cls}: {len(imgs)} görüntü")

                    for fn in imgs:
                        p = os.path.join(folder, fn)
                        img = cv2.imread(p)
                        if img is None:
                            skipped[split_name] += 1
                            continue

                        if resize_to is not None:
                            img = cv2.resize(img, (resize_to, resize_to), interpolation=cv2.INTER_CUBIC)

                        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        rgb.flags.writeable = False
                        res = holistic.process(rgb)

                        feat = extract_features_from_results(res, mode=mode)
                        if feat.shape[0] != dim:
                            skipped[split_name] += 1
                            continue

                        X_list.append(feat)
                        y_list.append(ci)
                        used[split_name] += 1

            process_split("train", train_dir, Xtr, ytr)
            process_split("test",  test_dir,  Xte, yte)

    if len(Xtr) == 0 or len(Xte) == 0:
        raise RuntimeError("Hiç veri çıkmadı. resize_to=512 dene veya dataset yolunu kontrol et.")

    X_train = np.stack(Xtr, axis=0).astype(np.float32)
    y_train = np.array(ytr, dtype=np.int64)
    X_test  = np.stack(Xte, axis=0).astype(np.float32)
    y_test  = np.array(yte, dtype=np.int64)

    np.savez_compressed(
        out,
        X_train=X_train, y_train=y_train,
        X_test=X_test,   y_test=y_test,
        classes=np.array(classes),
        mode=mode
    )

    print("\nSaved:", out)
    print("Train:", X_train.shape, "Test:", X_test.shape)
    print("Used:", used, "Skipped:", skipped)


5) Balanced batch sampler

In [34]:
def balanced_batch_indices(y, batch_size, rng):
    classes = np.unique(y)
    per = max(1, batch_size // len(classes))
    idxs = []
    for c in classes:
        pool = np.where(y == c)[0]
        pick = rng.choice(pool, size=per, replace=True)
        idxs.append(pick)
    idx = np.concatenate(idxs)
    if idx.size < batch_size:
        extra = rng.choice(np.arange(len(y)), size=(batch_size - idx.size), replace=True)
        idx = np.concatenate([idx, extra])
    rng.shuffle(idx)
    return idx[:batch_size]


6) MLP Base + SGD + Adam (Adam fixli)

In [35]:
# -----------------------------
# Base MLP (2-layer)
# -----------------------------
class BaseMLP:
    def __init__(self, in_dim, hidden, out_dim, seed=42, weight_decay=1e-4):
        rng = np.random.default_rng(seed)
        self.W1 = rng.normal(0, 0.2, size=(in_dim, hidden)).astype(np.float32)
        self.b1 = np.zeros((1, hidden), dtype=np.float32)
        self.W2 = rng.normal(0, 0.2, size=(hidden, out_dim)).astype(np.float32)
        self.b2 = np.zeros((1, out_dim), dtype=np.float32)

        self.weight_decay = float(weight_decay)

        self.loss_hist = []
        self.acc_hist = []
        self.wdelta_hist = []
        self.step_hist = []

    def forward(self, X):
        self.z1 = X @ self.W1 + self.b1
        # ReLU tanh’a göre genelde daha iyi
        self.a1 = np.maximum(self.z1, 0.0)
        self.z2 = self.a1 @ self.W2 + self.b2
        self.a2 = softmax(self.z2)
        return self.a2

    def predict(self, X):
        p = self.forward(X)
        return np.argmax(p, axis=1)

    def _grads(self, Xb, Yb, class_w=None):
        m = Xb.shape[0]
        y_pred = self.forward(Xb)
        loss = weighted_cross_entropy(y_pred, Yb, class_w=class_w)

        # weighted gradient
        if class_w is not None:
            sw = np.sum(Yb * class_w.reshape(1, -1), axis=1).reshape(-1, 1)
        else:
            sw = 1.0

        dz2 = (y_pred - Yb) * (sw / m)
        dW2 = self.a1.T @ dz2 + self.weight_decay * self.W2
        db2 = np.sum(dz2, axis=0, keepdims=True)

        da1 = dz2 @ self.W2.T
        dz1 = da1 * (self.z1 > 0).astype(np.float32)
        dW1 = Xb.T @ dz1 + self.weight_decay * self.W1
        db1 = np.sum(dz1, axis=0, keepdims=True)

        return float(loss), (dW1, db1, dW2, db2)

    def _wvec(self):
        return np.concatenate([self.W1.ravel(), self.b1.ravel(), self.W2.ravel(), self.b2.ravel()]).astype(np.float32)

def clip_by_norm(G, max_norm=5.0, eps=1e-9):
    n = float(np.sqrt(np.sum(G*G)) + eps)
    if n > max_norm:
        return G * (max_norm / n)
    return G

class MLP_SGD(BaseMLP):
    def __init__(self, in_dim, hidden, out_dim, lr=0.03, seed=42, weight_decay=1e-4):
        super().__init__(in_dim, hidden, out_dim, seed=seed, weight_decay=weight_decay)
        self.lr = float(lr)

    def train_epoch(self, X, Y, class_w=None, batch_size=128, seed=0):
        w0 = self._wvec()
        rng = np.random.default_rng(seed)
        idx = rng.permutation(X.shape[0])

        total_loss = 0.0
        step_sum = 0.0
        steps = 0

        for s in range(0, len(idx), batch_size):
            b = idx[s:s+batch_size]
            Xb, Yb = X[b], Y[b]
            loss, (dW1, db1, dW2, db2) = self._grads(Xb, Yb, class_w=class_w)

            dW1 = clip_by_norm(dW1, 5.0); dW2 = clip_by_norm(dW2, 5.0)

            uW1 = self.lr * dW1; ub1 = self.lr * db1
            uW2 = self.lr * dW2; ub2 = self.lr * db2

            self.W1 -= uW1; self.b1 -= ub1
            self.W2 -= uW2; self.b2 -= ub2

            total_loss += loss * Xb.shape[0]
            step_sum += (l2(uW1) + l2(ub1) + l2(uW2) + l2(ub2))
            steps += 1

        w1 = self._wvec()
        self.wdelta_hist.append(l2(w1 - w0))
        self.step_hist.append(step_sum / max(1, steps))
        return total_loss / X.shape[0]

class MLP_Adam(BaseMLP):
    def __init__(self, in_dim, hidden, out_dim, lr=0.0015, beta1=0.9, beta2=0.999, eps=1e-8, seed=42, weight_decay=1e-4):
        super().__init__(in_dim, hidden, out_dim, seed=seed, weight_decay=weight_decay)
        self.lr = float(lr); self.beta1=float(beta1); self.beta2=float(beta2); self.eps=float(eps)
        self.t = 0
        self.mW1 = np.zeros_like(self.W1); self.vW1 = np.zeros_like(self.W1)
        self.mb1 = np.zeros_like(self.b1); self.vb1 = np.zeros_like(self.b1)
        self.mW2 = np.zeros_like(self.W2); self.vW2 = np.zeros_like(self.W2)
        self.mb2 = np.zeros_like(self.b2); self.vb2 = np.zeros_like(self.b2)

    def _upd(self, P, G, M, V):
        self.t += 1
        M[:] = self.beta1 * M + (1 - self.beta1) * G
        V[:] = self.beta2 * V + (1 - self.beta2) * (G * G)
        Mh = M / (1 - self.beta1 ** self.t)
        Vh = V / (1 - self.beta2 ** self.t)
        U = self.lr * Mh / (np.sqrt(Vh) + self.eps)
        P[:] = P - U
        return U

    def train_epoch(self, X, Y, class_w=None, batch_size=128, seed=0):
        w0 = self._wvec()
        rng = np.random.default_rng(seed)
        idx = rng.permutation(X.shape[0])

        total_loss = 0.0
        step_sum = 0.0
        steps = 0

        for s in range(0, len(idx), batch_size):
            b = idx[s:s+batch_size]
            Xb, Yb = X[b], Y[b]
            loss, (dW1, db1, dW2, db2) = self._grads(Xb, Yb, class_w=class_w)

            dW1 = clip_by_norm(dW1, 5.0); dW2 = clip_by_norm(dW2, 5.0)

            uW1 = self._upd(self.W1, dW1, self.mW1, self.vW1)
            ub1 = self._upd(self.b1, db1, self.mb1, self.vb1)
            uW2 = self._upd(self.W2, dW2, self.mW2, self.vW2)
            ub2 = self._upd(self.b2, db2, self.mb2, self.vb2)

            total_loss += loss * Xb.shape[0]
            step_sum += (l2(uW1) + l2(ub1) + l2(uW2) + l2(ub2))
            steps += 1

        w1 = self._wvec()
        self.wdelta_hist.append(l2(w1 - w0))
        self.step_hist.append(step_sum / max(1, steps))
        return total_loss / X.shape[0]


7) KarcıFANN + Momentum-Karcı (ABS YOK, önceki pozitif ağırlık ref)

In [36]:
class PositiveRef:
    def __init__(self, W, w_floor=1e-2):
        self.w_floor = float(w_floor)
        self.ref = np.maximum(np.abs(W).astype(np.float32), self.w_floor)

    def update(self, W):
        W = W.astype(np.float32)
        pos = W > 0
        self.ref[pos] = W[pos]
        self.ref = np.maximum(self.ref, self.w_floor)
        return self.ref

# -----------------------------
# Karcı scale (math same, ref fixed)
# -----------------------------
def karci_scale(grad, W_ref_pos, loss_scalar, alpha, eps=1e-3, clip=0.20, r_clip=(1e-3, 1e3)):
    """
    SAME math:
      num = (loss+eps)^(alpha-1)
      den = (W_ref+eps)^(alpha-1)
      u = (num/den) * grad

    Stability:
      - W_ref_pos forced positive (prev-positive logic outside)
      - ratio clipping to avoid explosions when alpha<1 or alpha>>1
    """
    p = float(alpha - 1.0)
    loss = float(loss_scalar)
    num = (loss + eps)
    den = (W_ref_pos + eps)

    # ratio^p
    ratio = np.clip(num / den, r_clip[0], r_clip[1])
    scale = ratio ** p
    u = scale * grad

    if clip is not None:
        u = np.clip(u, -clip, clip)
    return u

class MLP_KarciFANN(BaseMLP):
    def __init__(self, in_dim, hidden, out_dim, alpha=1.2, clip=0.10, eps=1e-3, seed=42, weight_decay=1e-4):
        super().__init__(in_dim, hidden, out_dim, seed=seed, weight_decay=weight_decay)
        self.alpha = float(alpha)
        self.clip = float(clip)
        self.eps = float(eps)

        # hocanın dediği mantık: negatif olursa önceki pozitif referans
        self.W1_ref = np.abs(self.W1).astype(np.float32) + self.eps
        self.W2_ref = np.abs(self.W2).astype(np.float32) + self.eps

    def _update_refs(self):
        self.W1_ref = np.where(self.W1 > 0, self.W1, self.W1_ref)
        self.W2_ref = np.where(self.W2 > 0, self.W2, self.W2_ref)

    def train_epoch(self, X, Y, class_w=None, batch_size=128, seed=0):
        w0 = self._wvec()
        rng = np.random.default_rng(seed)
        idx = rng.permutation(X.shape[0])

        total_loss = 0.0
        step_sum = 0.0
        steps = 0

        for s in range(0, len(idx), batch_size):
            b = idx[s:s+batch_size]
            Xb, Yb = X[b], Y[b]
            loss, (dW1, db1, dW2, db2) = self._grads(Xb, Yb, class_w=class_w)

            dW1 = clip_by_norm(dW1, 5.0); dW2 = clip_by_norm(dW2, 5.0)

            # W_ref positive guaranteed
            W1_pos = np.maximum(self.W1_ref, self.eps).astype(np.float32)
            W2_pos = np.maximum(self.W2_ref, self.eps).astype(np.float32)
            b1_pos = np.maximum(np.abs(self.b1), self.eps).astype(np.float32)
            b2_pos = np.maximum(np.abs(self.b2), self.eps).astype(np.float32)

            uW1 = karci_scale(dW1, W1_pos, loss, self.alpha, eps=self.eps, clip=self.clip)
            ub1 = karci_scale(db1, b1_pos, loss, self.alpha, eps=self.eps, clip=self.clip)
            uW2 = karci_scale(dW2, W2_pos, loss, self.alpha, eps=self.eps, clip=self.clip)
            ub2 = karci_scale(db2, b2_pos, loss, self.alpha, eps=self.eps, clip=self.clip)

            self.W1 -= uW1; self.b1 -= ub1
            self.W2 -= uW2; self.b2 -= ub2

            self._update_refs()

            total_loss += loss * Xb.shape[0]
            step_sum += (l2(uW1) + l2(ub1) + l2(uW2) + l2(ub2))
            steps += 1

        w1 = self._wvec()
        self.wdelta_hist.append(l2(w1 - w0))
        self.step_hist.append(step_sum / max(1, steps))
        return total_loss / X.shape[0]

class MLP_Momentum_Karci(BaseMLP):
    def __init__(self, in_dim, hidden, out_dim, alpha=1.25, beta=0.85, clip=0.10, eps=1e-3, seed=42, weight_decay=1e-4):
        super().__init__(in_dim, hidden, out_dim, seed=seed, weight_decay=weight_decay)
        self.alpha=float(alpha); self.beta=float(beta)
        self.clip=float(clip); self.eps=float(eps)

        self.vW1 = np.zeros_like(self.W1); self.vb1 = np.zeros_like(self.b1)
        self.vW2 = np.zeros_like(self.W2); self.vb2 = np.zeros_like(self.b2)

        self.W1_ref = np.abs(self.W1).astype(np.float32) + self.eps
        self.W2_ref = np.abs(self.W2).astype(np.float32) + self.eps

    def _update_refs(self):
        self.W1_ref = np.where(self.W1 > 0, self.W1, self.W1_ref)
        self.W2_ref = np.where(self.W2 > 0, self.W2, self.W2_ref)

    def train_epoch(self, X, Y, class_w=None, batch_size=128, seed=0):
        w0 = self._wvec()
        rng = np.random.default_rng(seed)
        idx = rng.permutation(X.shape[0])

        total_loss = 0.0
        step_sum = 0.0
        steps = 0

        for s in range(0, len(idx), batch_size):
            b = idx[s:s+batch_size]
            Xb, Yb = X[b], Y[b]
            loss, (dW1, db1, dW2, db2) = self._grads(Xb, Yb, class_w=class_w)

            dW1 = clip_by_norm(dW1, 5.0); dW2 = clip_by_norm(dW2, 5.0)

            W1_pos = np.maximum(self.W1_ref, self.eps).astype(np.float32)
            W2_pos = np.maximum(self.W2_ref, self.eps).astype(np.float32)
            b1_pos = np.maximum(np.abs(self.b1), self.eps).astype(np.float32)
            b2_pos = np.maximum(np.abs(self.b2), self.eps).astype(np.float32)

            gW1 = karci_scale(dW1, W1_pos, loss, self.alpha, eps=self.eps, clip=self.clip)
            gb1 = karci_scale(db1, b1_pos, loss, self.alpha, eps=self.eps, clip=self.clip)
            gW2 = karci_scale(dW2, W2_pos, loss, self.alpha, eps=self.eps, clip=self.clip)
            gb2 = karci_scale(db2, b2_pos, loss, self.alpha, eps=self.eps, clip=self.clip)

            self.vW1 = self.beta * self.vW1 + gW1
            self.vb1 = self.beta * self.vb1 + gb1
            self.vW2 = self.beta * self.vW2 + gW2
            self.vb2 = self.beta * self.vb2 + gb2

            self.W1 -= self.vW1; self.b1 -= self.vb1
            self.W2 -= self.vW2; self.b2 -= self.vb2

            self._update_refs()

            total_loss += loss * Xb.shape[0]
            step_sum += (l2(self.vW1) + l2(self.vb1) + l2(self.vW2) + l2(self.vb2))
            steps += 1

        w1 = self._wvec()
        self.wdelta_hist.append(l2(w1 - w0))
        self.step_hist.append(step_sum / max(1, steps))
        return total_loss / X.shape[0]




8) train_compare (final)

In [37]:
# -----------------------------
# Train + Compare (early stop + best weights)
# -----------------------------
def train_compare(data_npz, epochs=120, hidden=512, batch=128, seed=42, save_model="fer_model_best.npz"):
    d = np.load(data_npz, allow_pickle=True)
    X_train = d["X_train"].astype(np.float32)
    y_train = d["y_train"].astype(np.int64)
    X_test  = d["X_test"].astype(np.float32)
    y_test  = d["y_test"].astype(np.int64)
    classes = d["classes"].astype(str)

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train).astype(np.float32)
    X_test_s  = scaler.transform(X_test).astype(np.float32)

    num_classes = len(classes)
    Y_train_oh = onehot(y_train, num_classes)

    class_w = compute_class_weights(y_train, num_classes)
    print("Train:", X_train_s.shape, "Test:", X_test_s.shape, "Classes:", classes)
    print("Class weights:", np.round(class_w, 3))

    in_dim = X_train_s.shape[1]

    # alpha aralığı: 0.8 - 1.8
    # pratikte FaceMesh + bu MLP için iyi başlangıçlar:
    models = [
        ("SGD", MLP_SGD(in_dim, hidden, num_classes, lr=0.03, seed=seed, weight_decay=2e-4)),
        ("Adam", MLP_Adam(in_dim, hidden, num_classes, lr=0.0015, seed=seed, weight_decay=2e-4)),
        ("KarcıFANN", MLP_KarciFANN(in_dim, hidden, num_classes, alpha=1.25, clip=0.10, eps=1e-3, seed=seed, weight_decay=2e-4)),
        ("Momentum-Karcı", MLP_Momentum_Karci(in_dim, hidden, num_classes, alpha=1.30, beta=0.85, clip=0.10, eps=1e-3, seed=seed, weight_decay=2e-4)),
    ]

    best_global = (-1.0, None, None)  # (acc, name, state)

    for name, m in models:
        best_acc = -1.0
        best_ep = 0
        best_state = None

        for ep in range(1, epochs + 1):
            loss = m.train_epoch(X_train_s, Y_train_oh, class_w=class_w, batch_size=batch, seed=seed + ep)
            pred = m.predict(X_test_s)
            acc = accuracy_score(y_test, pred)

            m.loss_hist.append(loss)
            m.acc_hist.append(float(acc))

            if acc > best_acc:
                best_acc = float(acc)
                best_ep = ep
                best_state = (m.W1.copy(), m.b1.copy(), m.W2.copy(), m.b2.copy())

            if ep % 10 == 0:
                print(f"[{name}] ep={ep:3d} loss={loss:.4f} acc={acc:.3f} | best={best_acc:.3f} @ep{best_ep}")

        # restore best for fair plotting & saving
        if best_state is not None:
            m.W1[:], m.b1[:], m.W2[:], m.b2[:] = best_state

        print(f"--- {name} best acc={best_acc:.3f} @ep{best_ep} ---")

        if best_acc > best_global[0]:
            best_global = (best_acc, name, best_state)

    # plots
    ep_axis = np.arange(1, epochs + 1)
    plt.figure(figsize=(16, 10))

    plt.subplot(2, 2, 1)
    for name, m in models:
        plt.plot(ep_axis, m.loss_hist, label=name)
    plt.title("Loss Karşılaştırma"); plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend()

    plt.subplot(2, 2, 2)
    for name, m in models:
        plt.plot(ep_axis, m.acc_hist, label=name)
    plt.title("Test Accuracy Karşılaştırma"); plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend()

    plt.subplot(2, 2, 3)
    for name, m in models:
        if len(m.wdelta_hist) == len(ep_axis):
            plt.plot(ep_axis, m.wdelta_hist, label=name)
    plt.title("Yakınsama: ||W_t-W_{t-1}||"); plt.xlabel("Epoch"); plt.ylabel("Norm"); plt.legend()

    plt.subplot(2, 2, 4)
    for name, m in models:
        if len(m.step_hist) == len(ep_axis):
            plt.plot(ep_axis, m.step_hist, label=name)
    plt.title("Adım Büyüklüğü"); plt.xlabel("Epoch"); plt.ylabel("Avg ||update||"); plt.legend()

    plt.tight_layout()
    plt.show()

    best_acc, best_name, best_state = best_global
    print("Best model:", best_name, "best acc:", best_acc)

    # confusion matrix for best model
    best_model = None
    for nm, m in models:
        if nm == best_name:
            best_model = m
            break
    pred = best_model.predict(X_test_s)
    cm = confusion_matrix(y_test, pred)
    print("Confusion Matrix:\n", cm)

    np.savez_compressed(
        save_model,
        model_name=best_name,
        W1=best_model.W1, b1=best_model.b1, W2=best_model.W2, b2=best_model.b2,
        mean=scaler.mean_.astype(np.float32),
        scale=scaler.scale_.astype(np.float32),
        classes=classes,
        mode="face_only",
        hidden=hidden
    )
    print("Saved model:", save_model)



9) Model load + webcam

In [2]:
def load_model_npz(model_npz):
    d = np.load(model_npz, allow_pickle=True)
    classes = d["classes"].astype(str)
    mode = str(d["mode"])
    W1 = d["W1"].astype(np.float32); b1 = d["b1"].astype(np.float32)
    W2 = d["W2"].astype(np.float32); b2 = d["b2"].astype(np.float32)
    mean = d["mean"].astype(np.float32)
    scale = d["scale"].astype(np.float32)
    return classes, mode, (W1, b1, W2, b2), mean, scale

def forward_np(X, params):
    W1, b1, W2, b2 = params
    z1 = X @ W1 + b1
    a1 = np.tanh(z1)
    z2 = a1 @ W2 + b2
    return softmax(z2)

def webcam_demo(model_npz, cam_index=0, resize_to=384):
    classes, mode, params, mean, scale = load_model_npz(model_npz)
    dim = expected_feature_dim(mode)

    cap = cv2.VideoCapture(cam_index)
    if not cap.isOpened():
        raise RuntimeError("Webcam açılamadı.")

    preds = []

    if mode == "face_only":
        with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.2) as face_det, \
             mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1,
                                   refine_landmarks=False,
                                   min_detection_confidence=0.2,
                                   min_tracking_confidence=0.2) as face_mesh:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                feat = facemesh_features_from_bgr(face_det, face_mesh, frame, resize_to=resize_to)
                if feat is not None and feat.shape[0] == dim:
                    x = feat.reshape(1, -1).astype(np.float32)
                    x = (x - mean) / (scale + 1e-8)
                    p = forward_np(x, params)[0]
                    cls_idx = int(np.argmax(p))
                    cls_name = classes[cls_idx]
                    conf = float(np.max(p))
                    preds.append(cls_name)
                    stable = majority_vote(np.array(preds, dtype=object), k=15)
                    text = f"{cls_name} ({conf:.2f}) | stable: {stable}"
                else:
                    text = "No face landmarks"

                cv2.rectangle(frame, (0, 0), (620, 60), (0, 0, 0), -1)
                cv2.putText(frame, text, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
                cv2.imshow("KarcıFANN Live", frame)

                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
    else:
        with mp_holistic.Holistic(static_image_mode=False, model_complexity=1,
                                  refine_face_landmarks=True,
                                  min_detection_confidence=0.5,
                                  min_tracking_confidence=0.5) as holistic:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                if resize_to is not None:
                    frame = cv2.resize(frame, (resize_to, resize_to), interpolation=cv2.INTER_CUBIC)

                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                rgb.flags.writeable = False
                res = holistic.process(rgb)

                feat = extract_features_from_results(res, mode=mode)
                if feat.shape[0] == dim:
                    x = feat.reshape(1, -1).astype(np.float32)
                    x = (x - mean) / (scale + 1e-8)
                    p = forward_np(x, params)[0]
                    cls_idx = int(np.argmax(p))
                    cls_name = classes[cls_idx]
                    conf = float(np.max(p))
                    preds.append(cls_name)
                    stable = majority_vote(np.array(preds, dtype=object), k=15)
                    text = f"{cls_name} ({conf:.2f}) | stable: {stable}"
                else:
                    text = "No landmarks"

                cv2.rectangle(frame, (0, 0), (620, 60), (0, 0, 0), -1)
                cv2.putText(frame, text, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
                cv2.imshow("KarcıFANN Live", frame)

                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break

    cap.release()
    cv2.destroyAllWindows()


10) Çalıştırma hücreleri

10.a Extract

In [39]:
DATASET_ROOT = r".\datasetKaggleFER2013"
# 1) Extract (align edilmiş)
extract_fer2013_folder(
    root=DATASET_ROOT,
    out="fer_mp_align.npz",
    resize_to=384
)


Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
[train] angry: 3995 görüntü
[train] disgust: 436 görüntü
[train] fear: 4097 görüntü
[train] happy: 7215 görüntü
[train] neutral: 4965 görüntü
[train] sad: 4830 görüntü
[train] surprise: 3171 görüntü
[test] angry: 958 görüntü
[test] disgust: 111 görüntü
[test] fear: 1024 görüntü
[test] happy: 1774 görüntü
[test] neutral: 1233 görüntü
[test] sad: 1247 görüntü
[test] surprise: 831 görüntü

Saved: fer_mp_align.npz
Train: (26386, 1872) Test: (6583, 1872)
Used: {'train': 26386, 'test': 6583} Skipped: {'train': 2323, 'test': 595}


10.b Train

In [ ]:
# 2) Train+Compare
train_compare(
    data_npz="fer_mp_align.npz",
    epochs=150,
    hidden=512,
    batch=128,
    save_model="fer_model_best.npz"
)

Train: (26386, 1872) Test: (6583, 1872) Classes: ['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']
Class weights: [0.491 4.494 0.456 0.244 0.357 0.395 0.562]
[SGD] ep= 10 loss=1.3241 acc=0.347 | best=0.379 @ep8
[SGD] ep= 20 loss=0.8475 acc=0.358 | best=0.433 @ep19
[SGD] ep= 30 loss=0.6619 acc=0.361 | best=0.464 @ep29
[SGD] ep= 40 loss=0.6131 acc=0.365 | best=0.464 @ep29
[SGD] ep= 50 loss=0.5829 acc=0.438 | best=0.480 @ep45
[SGD] ep= 60 loss=0.5766 acc=0.402 | best=0.480 @ep45
[SGD] ep= 70 loss=0.5643 acc=0.450 | best=0.480 @ep45
[SGD] ep= 80 loss=0.5565 acc=0.416 | best=0.487 @ep79
[SGD] ep= 90 loss=0.5532 acc=0.413 | best=0.487 @ep79
[SGD] ep=100 loss=0.5540 acc=0.445 | best=0.487 @ep79
[SGD] ep=110 loss=0.5370 acc=0.408 | best=0.487 @ep79
[SGD] ep=120 loss=0.5376 acc=0.477 | best=0.487 @ep79
[SGD] ep=130 loss=0.5323 acc=0.487 | best=0.487 @ep130
[SGD] ep=140 loss=0.5313 acc=0.432 | best=0.502 @ep133
[SGD] ep=150 loss=0.5309 acc=0.439 | best=0.502 @ep133
--- SGD best acc=0

10.c Webcam

In [ ]:
webcam_demo("fer_model_best.npz", cam_index=0, resize_to=384)

2.Entegre edilmiş versiyonu